In [1]:
import math
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools
import string

from toydown import GeoUnit, ToyDown

In [2]:
def create_demo_tree_homogenous_branching(h=3):
    """ Creates a demo tree of height h <= 16, with homogenous branching of h.
        This demo tree has two population counts, and count that is the sum of them.
    """
    leaves = ["1" + "".join(a) for a in itertools.product(string.hexdigits[1:h+1], repeat=h-1)]
    leaves_counts = np.random.randint(1000, size=(len(leaves),2))
    leaves_attrs = np.insert(leaves_counts, 0, leaves_counts.sum(axis=1), axis=1)
    
    nodes = {i: j for i, j in zip(leaves, leaves_attrs)}
    
    for i in range(2, h+1):
        level_names = list(set(list(map(lambda s: s[:-(i-1)], leaves))))
        level_counts = [np.zeros(3)]*len(level_names)
        for node in level_names:
            nodes[node] = np.array([v for k, v in nodes.items() if k.startswith(node) and len(k) == h]).sum(axis=0)
        
    return [GeoUnit(k, k[:-1], v) if k != "1" else GeoUnit(k, None, v) for k, v in nodes.items()]

In [3]:
geounits = create_demo_tree_homogenous_branching(3)
geounits.reverse()
eps = 1
eps_split = [0.25, 0.25, 0.5]

In [4]:
model = ToyDown(geounits, 3, eps, eps_split)
model.show()

1
├── 11
│   ├── 111
│   ├── 112
│   └── 113
├── 12
│   ├── 121
│   ├── 122
│   └── 123
└── 13
    ├── 131
    ├── 132
    └── 133



In [5]:
## Constraints: 0 difference among partition bins
## number of children n, returns list of 0 diff constraints.
cons_0_diff = lambda n: [{'type': 'eq', 'fun': lambda x, i=i:  x[i] - np.sum([x[j] for j in range(i+1,i+3)])} 
                         for i in np.arange(n*3, step=3)]

In [10]:
model.noise_and_adjust(node_cons=None, opts={"maxiter": 500, "disp": True})

1
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 30.106670451180662
            Iterations: 59
            Function evaluations: 705
            Gradient evaluations: 59
11
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 14.092494300320777
            Iterations: 57
            Function evaluations: 684
            Gradient evaluations: 57
113
112
111
13
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 11.584952413812346
            Iterations: 55
            Function evaluations: 660
            Gradient evaluations: 55
133
132
131
12
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 16.551933958131144
            Iterations: 50
            Function evaluations: 594
            Gradient evaluations: 50
123
122
121


In [11]:
model.get_node("1").data.__dict__

{'name': '1',
 'parent': None,
 'attributes': array([10481,  4828,  5653]),
 'identifier': '1',
 'level': 0,
 'noised': array([10495.91223018,  4823.82836614,  5644.84935295]),
 'noise': array([14.91223018, -4.17163386, -8.15064705]),
 'noise_type': 'laplacian',
 'adjusted': array([10495.91223017,  4823.82836614,  5644.84935294]),
 'error': array([-14.91223017,   4.17163386,   8.15064706])}

In [12]:
model.get_node("12").data.__dict__

{'name': '12',
 'parent': '1',
 'attributes': array([3666, 1813, 1853]),
 'identifier': '12',
 'level': 1,
 'noised': array([3658.58936596, 1810.47239113, 1851.85489581]),
 'noise': array([-7.41063404, -2.52760887, -1.14510419]),
 'noise_type': 'laplacian',
 'adjusted': array([3670.9707428 , 1810.39271809, 1851.76167977]),
 'error': array([-4.9707428 ,  2.60728191,  1.23832023])}

In [13]:
model.get_node("123").data.__dict__

{'name': '123',
 'parent': '12',
 'attributes': array([1376,  379,  997]),
 'identifier': '123',
 'level': 2,
 'noised': array([1372.87401109,  381.45053934,  994.50883436]),
 'noise': array([-3.12598891,  2.45053934, -2.49116564]),
 'noise_type': 'laplacian',
 'adjusted': array([1377.14680752,  378.64101277,  996.07711968]),
 'error': array([-1.14680752,  0.35898723,  0.92288032])}